Basic imports and path management

In [142]:
import pathlib
import sys
import time
import glob
from tqdm import tqdm
import multiprocessing

root = pathlib.Path("../../").resolve()
sys.path.append(str(root.joinpath("src", "python_framework_v02")))
sys.path.append(str(root.joinpath("src", "python_framework_v01")))
sys.path.append(str(root.joinpath("src", "python_routing_v01")))
sys.path.append(".")
import nhd_io as nio
import compute_nhd_routing_SingleSeg as tr
import nhd_network_utilities_v01 as nnu
import nhd_reach_utilities as nru

custom_input_folder = root.joinpath("test", "input", "yaml")
custom_input_file = "florence_933020089_dt60.yaml"
run_pocono2_test = None


Read the primary data input from json

In [143]:
supernetwork_parameters = None
waterbody_parameters = None
if custom_input_file:
    (
        supernetwork_parameters,
        waterbody_parameters,
        forcing_parameters,
        restart_parameters,
        output_parameters,
        run_parameters,
    ) = nio.read_custom_input(custom_input_folder.joinpath(custom_input_file))

    break_network_at_waterbodies = run_parameters.get(
        "break_network_at_waterbodies", None
    )

    dt = run_parameters.get("dt", None)
    nts = run_parameters.get("nts", None)
    qts_subdivisions = run_parameters.get("qts_subdivisions", None)
    debuglevel = -1 * int(run_parameters.get("debuglevel", 0))
    verbose = run_parameters.get("verbose", None)
    showtiming = run_parameters.get("showtiming", None)
    percentage_complete = run_parameters.get("percentage_complete", None)
    do_network_analysis_only = run_parameters.get("do_network_analysis_only", None)
    assume_short_ts = run_parameters.get("assume_short_ts", None)
    parallel_compute = run_parameters.get("parallel_compute", None)
    cpu_pool = run_parameters.get("cpu_pool", None)
    sort_networks = run_parameters.get("sort_networks", None)

    csv_output = output_parameters.get("csv_output", None)
    nc_output_folder = output_parameters.get("nc_output_folder", None)

    qlat_const = forcing_parameters.get("qlat_const", None)
    qlat_input_file = forcing_parameters.get("qlat_input_file", None)
    qlat_input_folder = forcing_parameters.get("qlat_input_folder", None)
    qlat_file_pattern_filter = forcing_parameters.get("qlat_file_pattern_filter", None)
    qlat_file_index_col = forcing_parameters.get("qlat_file_index_col", None)
    qlat_file_value_col = forcing_parameters.get("qlat_file_value_col", None)

    wrf_hydro_channel_restart_file = restart_parameters.get(
        "wrf_hydro_channel_restart_file", None
    )
    wrf_hydro_channel_ID_crosswalk_file = restart_parameters.get(
        "wrf_hydro_channel_ID_crosswalk_file", None
    )
    wrf_hydro_channel_ID_crosswalk_file_field_name = restart_parameters.get(
        "wrf_hydro_channel_ID_crosswalk_file_field_name", None
    )
    wrf_hydro_channel_restart_upstream_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_upstream_flow_field_name", None
    )
    wrf_hydro_channel_restart_downstream_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_downstream_flow_field_name", None
    )
    wrf_hydro_channel_restart_depth_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_depth_flow_field_name", None
    )

    wrf_hydro_waterbody_restart_file = restart_parameters.get(
        "wrf_hydro_waterbody_restart_file", None
    )
    wrf_hydro_waterbody_ID_crosswalk_file = restart_parameters.get(
        "wrf_hydro_waterbody_ID_crosswalk_file", None
    )
    wrf_hydro_waterbody_ID_crosswalk_file_field_name = restart_parameters.get(
        "wrf_hydro_waterbody_ID_crosswalk_file_field_name", None
    )
    wrf_hydro_waterbody_crosswalk_filter_file = restart_parameters.get(
        "wrf_hydro_waterbody_crosswalk_filter_file", None
    )
    wrf_hydro_waterbody_crosswalk_filter_file_field_name = restart_parameters.get(
        "wrf_hydro_waterbody_crosswalk_filter_file_field_name", None
    )

# Any specific commandline arguments will override the file
# TODO: There are probably some pathological collisions that could
# arise from this ordering ... check these out.


/home/jacob.hreha/github/t-route/src/python_framework_v02/nhd_io.py:53: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [144]:
if run_pocono2_test:
    if verbose:
        print("running test case for Pocono_TEST2 domain")
    # Overwrite the following test defaults
    supernetwork = "Pocono_TEST2"
    break_network_at_waterbodies = False
    qts_subdivisions = 1  # change qts_subdivisions = 1 as  default
    dt = 300 / qts_subdivisions
    nts = 144 * qts_subdivisions
    csv_output = {"csv_output_folder": os.path.join(root, "test", "output", "text")}
    nc_output_folder = os.path.join(root, "test", "output", "text")
    # test 1. Take lateral flow from re-formatted wrf-hydro output from Pocono Basin simulation
    qlat_input_file = os.path.join(
        root, r"test/input/geo/PoconoSampleData2/Pocono_ql_testsamp1_nwm_mc.csv"
    )


In [145]:
if showtiming:
    program_start_time = time.time()
if verbose:
    print(f"begin program t-route ...")

# STEP 1: Read the supernetwork dataset and build the connections graph
if verbose:
    print("creating supernetwork connections set")
if showtiming:
    start_time = time.time()

if supernetwork_parameters:
    supernetwork_values = nnu.get_nhd_connections(
        supernetwork_parameters=supernetwork_parameters,
        verbose=False,
        debuglevel=debuglevel,
    )

else:
    test_folder = os.path.join(root, r"test")
    geo_input_folder = os.path.join(test_folder, r"input", r"geo")
    supernetwork_parameters, supernetwork_values = nnu.set_networks(
        supernetwork=supernetwork,
        geo_input_folder=geo_input_folder,
        verbose=False,
        debuglevel=debuglevel,
    )
    waterbody_parameters = nnu.set_waterbody_parameters(
        supernetwork=supernetwork,
        geo_input_folder=geo_input_folder,
        verbose=False,
        debuglevel=debuglevel,
    )

if verbose:
    print("supernetwork connections set complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))

connections = supernetwork_values[0]


begin program t-route ...
creating supernetwork connections set
supernetwork connections set complete
... in 0.044088125228881836 seconds.


In [146]:
# STEP 2: Separate the networks and build the sub-graph of reaches within each network
if showtiming:
    start_time = time.time()
if verbose:
    print("organizing connections into reaches ...")
networks = nru.compose_networks(
    supernetwork_values,
    break_network_at_waterbodies=break_network_at_waterbodies,
    verbose=False,
    debuglevel=debuglevel,
    showtiming=showtiming,
)

if verbose:
    print("reach organization complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


organizing connections into reaches ...
reach organization complete
... in 0.006093502044677734 seconds.


In [147]:
# STEP 3: Organize Network for Waterbodies
if break_network_at_waterbodies:
    if showtiming:
        start_time = time.time()
    if verbose:
        print("reading waterbody parameter file ...")

    ## STEP 3a: Read waterbody parameter file
    waterbodies_values = supernetwork_values[12]
    waterbodies_segments = supernetwork_values[13]
    connections_tailwaters = supernetwork_values[4]

    waterbodies_df = nio.read_waterbody_df(waterbody_parameters, waterbodies_values,)
    waterbodies_df = waterbodies_df.sort_index(axis="index").sort_index(axis="columns")

    nru.order_networks(connections, networks, connections_tailwaters)

    if verbose:
        print("waterbodies complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()

    ## STEP 3b: Order subnetworks above and below reservoirs
    if showtiming:
        start_time = time.time()
    if verbose:
        print("ordering waterbody subnetworks ...")

    max_network_seqorder = -1
    for network in networks:
        max_network_seqorder = max(
            networks[network]["network_seqorder"], max_network_seqorder
        )
    ordered_networks = {}

    for terminal_segment, network in networks.items():
        if network["network_seqorder"] not in ordered_networks:
            ordered_networks[network["network_seqorder"]] = []
        ordered_networks[network["network_seqorder"]].append(
            (terminal_segment, network)
        )

    if verbose:
        print("ordering waterbody subnetworks complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()

else:
    # If we are not splitting the networks, we can put them all in one order
    max_network_seqorder = 0
    ordered_networks = {}
    ordered_networks[0] = [
        (terminal_segment, network) for terminal_segment, network in networks.items()
    ]

if do_network_analysis_only:
    sys.exit()

if break_network_at_waterbodies:
    ## STEP 3c: Handle Waterbody Initial States
    if showtiming:
        start_time = time.time()
    if verbose:
        print("setting waterbody initial states ...")

    if wrf_hydro_waterbody_restart_file:

        waterbody_initial_states_df = nio.get_reservoir_restart_from_wrf_hydro(
            wrf_hydro_waterbody_restart_file,
            wrf_hydro_waterbody_ID_crosswalk_file,
            wrf_hydro_waterbody_ID_crosswalk_file_field_name,
            wrf_hydro_waterbody_crosswalk_filter_file,
            wrf_hydro_waterbody_crosswalk_filter_file_field_name,
        )
    else:
        # TODO: Consider adding option to read cold state from route-link file
        waterbody_initial_ds_flow_const = 0.0
        waterbody_initial_depth_const = 0.0
        # Set initial states from cold-state
        waterbody_initial_states_df = pd.DataFrame(
            0, index=waterbodies_df.index, columns=["qd0", "h0",], dtype="float32"
        )
        # TODO: This assignment could probably by done in the above call
        waterbody_initial_states_df["qd0"] = waterbody_initial_ds_flow_const
        waterbody_initial_states_df["h0"] = waterbody_initial_depth_const
        waterbody_initial_states_df["index"] = range(len(waterbody_initial_states_df))

    if verbose:
        print("waterbody initial states complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()


reading waterbody parameter file ...
waterbodies complete
... in 0.014790058135986328 seconds.
ordering waterbody subnetworks ...
ordering waterbody subnetworks complete
... in 4.553794860839844e-05 seconds.
setting waterbody initial states ...
waterbody initial states complete
... in 0.019099950790405273 seconds.


In [148]:
# STEP 4: Handle Channel Initial States
if showtiming:
    start_time = time.time()
if verbose:
    print("setting channel initial states ...")

if wrf_hydro_channel_restart_file:

    channel_initial_states_df = nio.get_stream_restart_from_wrf_hydro(
        wrf_hydro_channel_restart_file,
        wrf_hydro_channel_ID_crosswalk_file,
        wrf_hydro_channel_ID_crosswalk_file_field_name,
        wrf_hydro_channel_restart_upstream_flow_field_name,
        wrf_hydro_channel_restart_downstream_flow_field_name,
        wrf_hydro_channel_restart_depth_flow_field_name,
    )
else:
    # TODO: Consider adding option to read cold state from route-link file
    channel_initial_us_flow_const = 0.0
    channel_initial_ds_flow_const = 0.0
    channel_initial_depth_const = 0.0
    # Set initial states from cold-state
    channel_initial_states_df = pd.DataFrame(
        0, index=connections.keys(), columns=["qu0", "qd0", "h0",], dtype="float32"
    )
    channel_initial_states_df["qu0"] = channel_initial_us_flow_const
    channel_initial_states_df["qd0"] = channel_initial_ds_flow_const
    channel_initial_states_df["h0"] = channel_initial_depth_const
    channel_initial_states_df["index"] = range(len(channel_initial_states_df))

if verbose:
    print("channel initial states complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


setting channel initial states ...
channel initial states complete
... in 0.020316600799560547 seconds.


In [149]:
# STEP 5: Read (or set) QLateral Inputs
if showtiming:
    start_time = time.time()
if verbose:
    print("creating qlateral array ...")

# initialize qlateral dict
qlateral = {}

if qlat_input_folder:
    qlat_files = []
    for pattern in qlat_file_pattern_filter:
        qlat_files.extend(glob.glob(qlat_input_folder + pattern))
    qlat_df = nio.get_ql_from_wrf_hydro(
        qlat_files=qlat_files,
        index_col=qlat_file_index_col,
        value_col=qlat_file_value_col,
    )

elif qlat_input_file:
    qlat_df = nio.get_ql_from_csv(qlat_input_file)

else:
    qlat_df = pd.DataFrame(
        qlat_const, index=connections.keys(), columns=range(nts), dtype="float32"
    )

for index, row in qlat_df.iterrows():
    qlateral[index] = row.tolist()

if verbose:
    print("qlateral array complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


creating qlateral array ...
qlateral array complete
... in 6.030196905136108 seconds.


In [150]:
# STEP 6: Sort the ordered networks
if sort_networks:
    if showtiming:
        start_time = time.time()
    if verbose:
        print("sorting the ordered networks ...")

    for nsq in range(max_network_seqorder, -1, -1):
        sort_ordered_network(ordered_networks[nsq], True)

    if verbose:
        print("sorting complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()


In [151]:
# Define the pool after we create the static global objects (and collect the garbage)
if parallel_compute:
    import gc

    gc.collect()
    pool = multiprocessing.Pool(cpu_pool)

flowveldepth_connect = (
    {}
)  # dict to contain values to transfer from upstream to downstream networks


writing netcdf output to --> ../../test/output/text/8777247.ncwriting netcdf output to --> ../../test/output/text/8777239.ncwriting netcdf output to --> ../../test/output/text/8777229.nc

writing netcdf output to --> ../../test/output/text/8777253.nc

writing netcdf output to --> ../../test/output/text/8777191.ncwriting netcdf output to --> ../../test/output/text/8777173.nc

writing netcdf output to --> ../../test/output/text/8777285.nc
writing netcdf output to --> ../../test/output/text/8778365.nc
writing netcdf output to --> ../../test/output/text/8777207.nc
writing netcdf output to --> ../../test/output/text/8777187.nc
writing netcdf output to --> ../../test/output/text/8777241.nc
writing netcdf output to --> ../../test/output/text/8777141.nc
writing netcdf output to --> ../../test/output/text/8777225.nc
writing netcdf output to --> ../../test/output/text/8777125.nc
writing netcdf output to --> ../../test/output/text/8778363.nc
writing netcdf output to --> ../../test/output/text/877

In [152]:
################### Main Execution Loop across ordered networks
if showtiming:
    main_start_time = time.time()
if verbose:
    print(f"executing routing computation ...")

compute_network_func = tr.compute_network

tr.connections = connections
tr.networks = networks
tr.qlateral = qlateral
tr.waterbodies_df = waterbodies_df
tr.waterbody_initial_states_df = waterbody_initial_states_df
tr.channel_initial_states_df = channel_initial_states_df

progress_count = 0
percentage_complete = True
if percentage_complete:
    for nsq in range(max_network_seqorder, -1, -1):
        for terminal_segment, network in ordered_networks[nsq]:
            progress_count += len(network["all_segments"])
    pbar = tqdm(total=(progress_count))

for nsq in range(max_network_seqorder, -1, -1):

    if parallel_compute:
        nslist = []
    results = []

    current_index_total = 0

    for terminal_segment, network in ordered_networks[nsq]:

        if percentage_complete:
            if current_index_total == 0:
                pbar.update(0)

        if break_network_at_waterbodies:
            waterbody = waterbodies_segments.get(terminal_segment)
        else:
            waterbody = None
        if not parallel_compute:  # serial execution
            if showtiming:
                start_time = time.time()
            if verbose:
                print(
                    f"routing ordered reaches for terminal segment {terminal_segment} ..."
                )

            results.append(
                compute_network_func(
                    flowveldepth_connect=flowveldepth_connect,
                    terminal_segment=terminal_segment,
                    supernetwork_parameters=supernetwork_parameters,
                    waterbody_parameters=waterbody_parameters,
                    waterbody=waterbody,
                    nts=nts,
                    dt=dt,
                    qts_subdivisions=qts_subdivisions,
                    verbose=verbose,
                    debuglevel=debuglevel,
                    csv_output=csv_output,
                    nc_output_folder=nc_output_folder,
                    assume_short_ts=assume_short_ts,
                )
            )

            if showtiming:
                print("... complete in %s seconds." % (time.time() - start_time))
            if percentage_complete:
                pbar.update(len(network["all_segments"]))

        else:  # parallel execution
            nslist.append(
                [
                    flowveldepth_connect,
                    terminal_segment,
                    supernetwork_parameters,  # TODO: This should probably be global...
                    waterbody_parameters,
                    waterbody,
                    nts,
                    dt,
                    qts_subdivisions,
                    verbose,
                    debuglevel,
                    csv_output,
                    nc_output_folder,
                    assume_short_ts,
                ]
            )

    if parallel_compute:
        if verbose:
            print(f"routing ordered reaches for networks of order {nsq} ... ")
        if debuglevel <= -2:
            print(f"reaches to be routed include:")
            print(f"{[network[0] for network in ordered_networks[nsq]]}")
        # with pool:
        # with multiprocessing.Pool() as pool:
        results = pool.starmap(compute_network_func, nslist)

        if showtiming:
            print("... complete in %s seconds." % (time.time() - start_time))
        if percentage_complete:
            pbar.update(
                sum(
                    len(network[1]["all_segments"]) for network in ordered_networks[nsq]
                )
            )
            # print(f"{[network[0] for network in ordered_networks[nsq]]}")

    max_courant = 0
    maxa = []
    for result in results:
        for seg in result:
            maxa.extend(result[seg][:, 8:9])
    max_courant = max(maxa)
    print(f"max_courant: {max_courant}")

    if (
        nsq > 0
    ):  # We skip this step for zero-order networks, i.e., those that have no downstream dependents
        flowveldepth_connect = (
            {}
        )  # There is no need to preserve previously passed on values -- so we clear the dictionary
        for i, (terminal_segment, network) in enumerate(ordered_networks[nsq]):
            # seg = network["reaches"][network["terminal_reach"]]["reach_tail"]
            seg = terminal_segment
            flowveldepth_connect[seg] = {}
            flowveldepth_connect[seg] = results[i][seg]
            # TODO: The value passed here could be much more specific to
            # TODO: exactly and only the most recent time step for the passing reach

if parallel_compute:
    pool.close()

if percentage_complete:
    pbar.close()

if verbose:
    print("ordered reach computation complete")
if showtiming:
    print("... in %s seconds." % (time.time() - main_start_time))
if verbose:
    print("program complete")
if showtiming:
    print("... in %s seconds." % (time.time() - program_start_time))


  0%|          | 0/1108 [00:00<?, ?it/s]

executing routing computation ...
routing ordered reaches for networks of order 2 ... 


 42%|████▏     | 467/1108 [00:59<01:21,  7.89it/s]

... complete in 60.87676024436951 seconds.
max_courant: [5.51265717]
routing ordered reaches for networks of order 1 ... 


 45%|████▌     | 499/1108 [01:23<03:12,  3.16it/s]

... complete in 85.1832218170166 seconds.
max_courant: [0.]
routing ordered reaches for networks of order 0 ... 


100%|██████████| 1108/1108 [03:02<00:00,  3.70it/s]

... complete in 183.91847133636475 seconds.


100%|██████████| 1108/1108 [03:07<00:00,  5.92it/s]

max_courant: [25.36383247]
ordered reach computation complete
... in 187.30772423744202 seconds.
program complete
... in 195.129958152771 seconds.


In [153]:
#creation of the main dataframe

In [154]:
# print(results)
import numpy as np
import itertools
import pandas as pd
all_results = {}
seg_courant_maxes = []
time = []
flowval = []  # flowval
velval_list = []  # velval
depthval = []  # depthval
qlatval = []  # qlatval
storageval = []  # storageval
qlatCumval = []  # qlatCumval
kinCelerity = [] # ck
courant = [] # cn
X = [] # X

for result in results:
    all_results.update(result)

# print(all_results[8780801][1][0])

df = pd.DataFrame()
# print(all_results)

for key , data in all_results.items():
    time.extend(data[:,0]) # time
    flowval.extend(data[:,1])  # flowval
    velval_list.extend(data[:,2]) # velval
    depthval.extend(data[:,3])  # depthval
    qlatval.extend(data[:,4])  # qlatval
    storageval.extend(data[:,5]) # storageval
    qlatCumval.extend(data[:,6])  # qlatCumval
    kinCelerity.extend(data[:,7]) # ck
    courant.extend(data[:,8]) # cn
    X.extend(data[:,9]) # X
    

In [155]:
single_seg_length = len(all_results[933020089][:,0])
single_seg_length*611

10998000

In [156]:
len(time)

10998000

In [157]:
key_index = all_results.keys()
key_index = list(itertools.chain.from_iterable(itertools.repeat(x, single_seg_length) for x in key_index))
data = {'key_index':list(key_index),
        'time':list(time),
        'flowval':list(flowval),
        'velval_list':list(velval_list),
        'depthval':list(depthval),
        'qlatval':list(qlatval),
        'storageval':list(storageval),
        'qlatCumval':list(qlatCumval),
        'kinCelerity':list(kinCelerity),
        'courant':list(courant),
        'X':list(X)}
df = pd.DataFrame(data)
df = df.set_index('key_index')  
df = df.reset_index()
df = df.set_index('key_index')  

df

,time,flowval,velval_list,depthval,qlatval,storageval,qlatCumval,kinCelerity,courant,X
key_index,,,,,,,,,,
8780801,0.0,0.000177,0.033756,0.004995,0.000177,6.984919e-09,1.062422e-02,0.055861,0.017548,0.499502
8780801,60.0,0.000177,0.033788,0.005002,0.000177,1.309672e-08,2.124843e-02,0.055914,0.017565,0.499503
8780801,120.0,0.000177,0.033795,0.005004,0.000177,1.833541e-08,3.187265e-02,0.055926,0.017568,0.499503
8780801,180.0,0.000177,0.033797,0.005004,0.000177,2.357410e-08,4.249686e-02,0.055929,0.017569,0.499503
8780801,240.0,0.000177,0.033797,0.005004,0.000177,2.793968e-08,5.312108e-02,0.055930,0.017569,0.499503
...,...,...,...,...,...,...,...,...,...,...
8778465,1079700.0,0.000033,0.000000,0.000000,0.133463,8.546982e+06,1.440267e+06,0.000000,0.000000,0.000000
8778465,1079760.0,0.000033,0.000000,0.000000,0.133463,8.547031e+06,1.440275e+06,0.000000,0.000000,0.000000
8778465,1079820.0,0.000033,0.000000,0.000000,0.133463,8.547080e+06,1.440283e+06,0.000000,0.000000,0.000000


In [158]:
!pip install plotly

In [159]:
#top 25 courants based on our work this morning and their segment IDs

In [160]:
seg_list = []
seg_courant_maxes = []
for seg in all_results:
    seg_list.append(seg)
    seg_courant_maxes.append(max(all_results[seg][:,tr.courant_index]))
zipped = zip(seg_list,seg_courant_maxes)
zipped = list(zipped)
res = sorted(zipped, key = lambda x: x[1],reverse=True) 
# A.sort(reverse=True)
res = (res)[:25]
print((res)[:25])


[(933020027, 25.363832473754883), (8777735, 14.950478553771973), (933020017, 9.564001083374023), (8778581, 8.574037551879883), (933020059, 7.254691123962402), (933020061, 4.696855068206787), (8780597, 3.591726064682007), (933020020, 2.8707876205444336), (933020058, 2.4734983444213867), (933020023, 2.1059963703155518), (8777485, 1.860101580619812), (933020036, 1.6248382329940796), (8778795, 1.6150635480880737), (8778021, 1.4462398290634155), (8777339, 1.393180251121521), (8777867, 1.353600263595581), (8777477, 1.3393661975860596), (8780495, 1.263046383857727), (8777861, 1.1583272218704224), (8777481, 1.149632215499878), (8780607, 1.0738232135772705), (933020050, 1.0441739559173584), (8779003, 0.9976801872253418), (8780751, 0.9917784929275513), (8778947, 0.9753884673118591)]


In [161]:
#A list of those IDs

In [162]:
major_segments = []
for x,y in res[:25]:
    major_segments.append(x)
major_segments

[933020027,
 8777735,
 933020017,
 8778581,
 933020059,
 933020061,
 8780597,
 933020020,
 933020058,
 933020023,
 8777485,
 933020036,
 8778795,
 8778021,
 8777339,
 8777867,
 8777477,
 8780495,
 8777861,
 8777481,
 8780607,
 933020050,
 8779003,
 8780751,
 8778947]

In [163]:
#Filtering out the above segment IDs from the original df.

In [164]:

df_filtered= df.loc[ df.index.isin(major_segments), : ]
df_filtered

,time,flowval,velval_list,depthval,qlatval,storageval,qlatCumval,kinCelerity,courant,X
key_index,,,,,,,,,,
8777735,0.0,0.004696,0.206769,0.011542,3.463928e-05,0.000002,0.002078,0.341748,1.864082,0.497108
8777735,60.0,0.004698,0.206816,0.011546,3.463928e-05,0.000003,0.004157,0.341824,1.864496,0.497108
8777735,120.0,0.004700,0.206857,0.011549,3.463928e-05,0.000003,0.006235,0.341891,1.864862,0.497107
8777735,180.0,0.004703,0.206897,0.011552,3.463928e-05,0.000004,0.008313,0.341956,1.865216,0.497106
8777735,240.0,0.004705,0.206935,0.011556,3.463928e-05,0.000004,0.010392,0.342019,1.865557,0.497106
...,...,...,...,...,...,...,...,...,...,...
8780751,1079700.0,0.006394,0.119810,0.024856,6.069202e-07,-15.666612,45.220530,0.196448,0.129526,0.494083
8780751,1079760.0,0.006394,0.119809,0.024855,6.069202e-07,-15.666626,45.220566,0.196447,0.129526,0.494083
8780751,1079820.0,0.006394,0.119809,0.024855,6.069202e-07,-15.666641,45.220603,0.196447,0.129525,0.494083


In [165]:
df.loc[df['courant'] == 25.363832473754883]

,time,flowval,velval_list,depthval,qlatval,storageval,qlatCumval,kinCelerity,courant,X
key_index,,,,,,,,,,
933020027,302400.0,15.181515,4.458032,1.180036,0.0,58.131263,0.0,4.227305,25.363832,0.40232


In [166]:
#Automated plotter based on filtered dataframe

In [167]:
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.io as pio
df_indexed = df_filtered.reset_index()
test_chart = go.FigureWidget()
for x in range(0,25):
    temp_df_range_1 = single_seg_length*(x)
    temp_df_range_2 = single_seg_length*(x+1)
    print(temp_df_range_1,temp_df_range_2)
    test_chart.add_scatter(x=df_indexed[temp_df_range_1:temp_df_range_2]['time'],
        y=df_indexed[temp_df_range_1:temp_df_range_2]['flowval'],
    name="segment " + str(df_indexed['key_index'][temp_df_range_1])
            );
test_chart.layout.title = "Timestep Chart " + str(dt)
plot(test_chart)

0 18000
18000 36000
36000 54000
54000 72000
72000 90000
90000 108000
108000 126000
126000 144000
144000 162000
162000 180000
180000 198000
198000 216000
216000 234000
234000 252000
252000 270000
270000 288000
288000 306000
306000 324000
324000 342000
342000 360000
360000 378000
378000 396000
396000 414000
414000 432000
432000 450000


'temp-plot.html'